In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.models import Model, load_model 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

In [ ]:
# load the saved model
model_final = load_model('models/ResNet50_trained_f2c_continue.h5')
model_final.summary()

In [ ]:
def print_layer_trainable(model_name):
    for layer in model_name.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
print_layer_trainable(model_final)

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model_final.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# load dataset
img_width, img_height = 224, 224
train_data_dir = "images/train_col224"
validation_data_dir = "images/dev_col224"
nb_train_samples = 32488
nb_validation_samples = 5723
batch_size = 32
epochs = 50

In [ ]:
# initiate the train and validation generators with data augumentation
train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# save the model according to the conditions
checkpoint = ModelCheckpoint("models/main_ResNet50_trained.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

In [ ]:
H = model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = 1,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = [checkpoint, early])

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = epochs
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")
plt.savefig('ResNet50_trained_f2c_continue.png')
plt.show()

In [ ]:
eval_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

eval_generator = eval_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
preds = model_final.evaluate_generator(
    eval_generator,
    steps = nb_validation_samples // batch_size)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
# plot_model(model, to_file='model.png')
SVG(model_to_dot(model_final).create(prog='dot', format='svg'))